In [48]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(42)

In [49]:
trn = pd.read_csv('./data/train_ver2.csv')
tst = pd.read_csv('./data/test_ver2.csv')

C:\anaconda3\envs\practice-ml\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\anaconda3\envs\practice-ml\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [50]:
# 제품 변수들의 결측값을 0으로 대체한다.
prods = trn.columns[24:].tolist()
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 모든 제품이 0인 행 제거
no_product = trn[prods].sum(axis=1) == 0
trn = trn[-no_product]

# train / test 데이터를 합친다. test 데이터의 제품 부분은 모두 0으로 채운다.
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

# 학습에 사용할 변수를 담는 리스트
features = []

# 번주형 변수들을 label encoding한다.
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes',
                    'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall',
                    'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols


# 수치형 변수들의 특이값과 결측값을 -99로 대체하고 int형으로 바꾼다.
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

# 학습에 사용할 수치형 변수를 추가한다.
features += ['age', 'antiguedad', 'renta', 'ind_nuevo', 'indrel',
             'indrel_1mes', 'ind_actividad_cliente']

## 피쳐 엔지니어링

In [51]:
# 두 날짜 변수에서 연도와 월 정보를 추출한다.
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float
                                              else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float
                                              else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float
                                              else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float
                                              else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

# 그 외 변수들의 결측값을 -99로 대체한다.
df.fillna(-99, inplace=True)

# lag - 1 데이터를 생성한다.

def date_to_int(str_data):
    Y, M, D = [int(a) for a in str_data.strip().split('-')]
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date']
                   else col for col in df.columns]
df_lag['int_date'] += 1

df_trn = df.merge(df_lag, on=['ncodpers', 'int_date'], how='left')

del df, df_lag

for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

## 모델 학습